In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
def preprocess(w):
    w = w.split(',')[0].strip()
    w = w.split('(')[0].strip()
    w = w.split('/')[0].strip()
    w = "_".join(list(map(lambda x:x.strip(), w.split('&'))))
    w = w.replace("St ","Saint ")
    w = w.replace("-"," ")
    w = w.replace("."," ")
    w = w.replace("*"," ")
    w = w.replace("\'"," ")
    w = w.split()
    if w[-1].lower() == 'airport': w = w[0:-1]
    if w[-1].lower() == 'intl': w = w[0:-1]
    w = "_".join(w)
    return w

In [3]:
df_aircraft = pd.read_csv('./data/df_aircraft.csv', sep='§', engine='python',
                          index_col=0, encoding='utf-8')
df_airline = pd.read_csv('./data/df_airline.csv', sep='§', engine='python',
                         index_col=0, encoding='utf-8')
df_airport = pd.read_csv('./data/df_airport.csv', sep='§', engine='python',
                         index_col=0, encoding='utf-8')
df_country = pd.read_csv('./data/df_country.csv', sep='§', engine='python',
                         index_col=0, encoding='utf-8')


In [4]:
tabs = ['default','bars']
cate = ['Seat', 'Food']
manu = ['Airbus', 'Boeing', 'Cessna', 'Cirrus ','Pilatus', 'Matra']
airl = list(df_airline['airline'].apply(preprocess))
airc = list(df_aircraft['model'].apply(preprocess))

In [5]:
CT_tabs = 'tab'
CT_sess = 'session'
CT_date = 'date'
CT_filt = 'filters'
CT_filt_manu = 'manufacturer'
CT_filt_airc = 'aircraft'
CT_filt_airl = 'company'
CT_filt_cate = 'category'
CT_filt_date = 'date'

In [6]:
def make_date() -> str :
    year = str(np.random.randint(2010,2020)).zfill(4)
    month = str(np.random.randint(12)).zfill(2)
    day = str(np.random.randint(31)).zfill(2)
    return day+month+year

def make_timestamp() -> str :
    year = str(np.random.randint(2010,2020)).zfill(4)
    month = str(np.random.randint(12)).zfill(2)
    day = str(np.random.randint(31)).zfill(2)
    hour = str(np.random.randint(24)).zfill(2)
    minute = str(np.random.randint(60)).zfill(2)
    sec = str(np.random.randint(60)).zfill(2)
    return day+'/'+month+'/'+year+' '+hour+'h'+minute+'m'+sec+'s'

def make_manu() -> str:
    return np.random.choice(manu,1)[0]

def make_airl() -> str:
    return np.random.choice(airl,1)[0]

def make_airc() -> str:
    return np.random.choice(airc,1)[0]

def make_cate() -> str:
    return np.random.choice(cate,1)[0]

def make_tab() -> str:
    return np.random.choice(tabs,1)[0]

def init_filters() -> dict:
    filters = {
        CT_filt_manu : [],
        CT_filt_airc : [],
        CT_filt_airl : [],
        CT_filt_cate : [],
        CT_filt_date : [],
    }
    return filters

def make_event_str(sessid: int,
                   timestamp: str,
                   tab: str = 'default',
                   filters : dict = None) -> str :
    
    if filters is None:
        filters = init_filters()
    
    event = {
        CT_sess : sessid,
        CT_filt : filters,
        CT_tabs : tab,
        CT_date : timestamp,
    }
    
    event_string = json.dumps(event)
    return event_string

def make_filters() -> dict:
    filters = {
        CT_filt_manu : [make_manu() for i in range(np.random.randint(5))],
        CT_filt_airc : [make_airc() for i in range(np.random.randint(5))],
        CT_filt_airl : [make_airl() for i in range(np.random.randint(5))],
        CT_filt_cate : [make_cate() for i in range(np.random.randint(5))],
        CT_filt_date : [make_date(), make_date()],
    }
    return filters

def make_session(sessid:int, n=None) -> pd.DataFrame:
    df = make_bdd(0)
    if n is None:
        n = np.random.randint(1,10)
    for i in range(n):
        timestamp = make_timestamp()
        filters = make_filters()
        tab = make_tab()
        event = make_event_str(sessid, timestamp, tab , filters)
        row = pd.Series([sessid, timestamp, event],
                        index = df.columns)
        df = df.append(row, ignore_index=True)
    return df

def make_bdd(nb_session : int = 1):
    bdd = pd.DataFrame(columns=['session','date','json'])
    for i in range(nb_session):
        bdd = bdd.append(make_session(i)).reset_index(drop=True)
    return bdd

In [7]:
bdd = make_bdd(5)
bdd.head()

,session,date,json
0,0,06/09/2014 04h21m18s,"{""session"": 0, ""filters"": {""manufacturer"": [""C..."
1,0,03/06/2017 20h17m36s,"{""session"": 0, ""filters"": {""manufacturer"": [""B..."
2,0,01/04/2015 19h54m17s,"{""session"": 0, ""filters"": {""manufacturer"": [""P..."
3,0,14/03/2012 05h15m45s,"{""session"": 0, ""filters"": {""manufacturer"": [""M..."
4,0,25/08/2013 15h05m22s,"{""session"": 0, ""filters"": {""manufacturer"": [""B..."


In [8]:
event_string = bdd.loc[0,'json']
event_dict = json.loads(event_string)
event_dict

{'session': 0,
 'filters': {'manufacturer': ['Cirrus ', 'Pilatus'],
  'aircraft': [],
  'company': ['Aeropostal_Alas_de_Venezuela',
   'Overland_Airways',
   'Era_Aviation',
   'Air_Alpha_Greenland'],
  'category': ['Seat'],
  'date': ['04092019', '29032010']},
 'tab': 'default',
 'date': '06/09/2014 04h21m18s'}